In [ ]:
# 载入所需库包
import datetime
import collections
import jieba
import re
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn import metrics
import pandas as pd

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning, DataConversionWarning, FitFailedWarning, NotFittedError
# 忽略所有的警告
warnings.filterwarnings('ignore')

In [ ]:
#1.读入 分词后的新闻标题、网址及文本 的 Excel档
print ('1.读入 分词后的新闻标题、网址及文本 的 Excel档' )
print('开始时间：' + str(datetime.datetime.now()))

news_df = pd.read_excel('data分词后.xlsx', header=0)

# 检查训练数据是否合规
categories = {'于是':0, '孤独患者':0, '罗生门':0}
i = 0; err = 0
while i < len(news_df):
    if len(str(news_df.分词后内容[i])) < 1: err = err + 1 # 文本少于10个字视为异常
    categories[news_df.歌名[i]] += 1 # 累计各类文本数量
    i = i + 1

# 打印出 各类训练数量、总数量 及 异常数量
print('')
print(categories) 
print('聚类数据数量：' + str(len(news_df)) + '，异常数据数量：' + str(err) 
      + '，结束时间：' + str(datetime.datetime.now()))

# print(news_df[:5])

news_df['聚类'] = ''
news_df['聚类用内容'] = ''

print('\n结束时间：' + str(datetime.datetime.now()))

In [ ]:
#2.抽取词向量特征
print ('2.抽取词向量特征' )
print('开始时间：' + str(datetime.datetime.now()))

docs = news_df['分词后内容'].tolist()  # 更简洁的列表推导式

# 设置特征数量限制
max_features = 10000  # 根据您的内存调整
vectorizer = TfidfVectorizer(max_df=0.7, max_features=max_features)
tfidf = vectorizer.fit_transform(docs)
 
word = vectorizer.get_feature_names_out()

print('tfidf.shape：' + str(tfidf.shape))
print('len(word)：' + str(len(word)))

weight = tfidf.toarray()       # 将 tf-idf 矩阵抽取出来
#元素w[i][j]表示j词在i类文本的 tf-idf 权重

print('tfidf.shape：' + str(tfidf.shape))
print('len(word)：' + str(len(word)))
print('len(weight)：' + str(len(weight)))

print('\n结束时间：' + str(datetime.datetime.now()))

In [ ]:
# 3.设置聚类个数，为了兼顾可解释性、有效性，可通过不同 k 值进行迭代比较
print ('3.设置聚类个数' )
print('开始时间：' + str(datetime.datetime.now()))

k = 3
kmeans = KMeans(n_clusters=k)
#kmini_model = kmini.fit(weight)
pca = PCA(n_components=1000)
newData = pca.fit_transform(weight)
kmeans.fit(weight)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
n_clusters=k, n_init=100, random_state=None, tol=0.001, verbose=0)
print('前十笔文本聚类结果：')
print(kmeans.labels_[:10])

print('\n结束时间：' + str(datetime.datetime.now()))

In [ ]:
# 4.各聚类的文本数量及常用词汇
print ('4.各聚类的文本数量及常用词汇' )
print('开始时间：' + str(datetime.datetime.now()))

cluster = [[], [], []]
for i in range(len(kmeans.labels_)):
    if kmeans.labels_[i]==0:
        cluster[0].append(docs[i])
    if kmeans.labels_[i]==1:
        cluster[1].append(docs[i])
    if kmeans.labels_[i]==2:
        cluster[2].append(docs[i])
    if kmeans.labels_[i]==3:
        cluster[3].append(docs[i])
    news_df.loc[i, '聚类'] = str(kmeans.labels_[i])
    news_df.loc[i, '聚类用内容'] = docs[i]

from collections import Counter

for i in range(3):
    print('cluster[' + str(i) + ']：' + str(len(cluster[i])) + ' 条评论，用词总数 及 词频前20名：')
    clustercontent = ' '.join(cluster[i])
#    print('clustercontent：' + str(len(clustercontent)))
    words = clustercontent.split()
#    print('words：' + str(len(words)))
    
    #取词频前20名的字词(改采 collections 库)
    word_freq = Counter(words)
    print(len(word_freq))
    print(word_freq.most_common(20))  
    print(' ')
    
print('结束时间：' + str(datetime.datetime.now()))

In [ ]:
# 5.将聚类后的 新闻 DataFrame 存入 Excel 档
print ('5.将聚类后的 新闻 DataFrame 存入 Excel 档' )
print('开始时间：' + str(datetime.datetime.now()))

news_df.to_excel('data聚类后.xlsx', header=True)

print('\n结束时间：' + str(datetime.datetime.now()))

In [ ]:
# 6.将聚类后的结果作成交叉分析表
print ('6.将聚类后的结果作成交叉分析表' )
print('开始时间：' + str(datetime.datetime.now()))
print(' ')

print(pd.crosstab(news_df['歌名'], news_df['聚类'], margins=True, margins_name='合计'))

print('\n结束时间：' + str(datetime.datetime.now()))